In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import yfinance

In [2]:
ticker = "INFY.NS"

In [3]:
def get_historical_stock_price(ticker,period='1d',interval='5m'):
    ticker = yfinance.Ticker(ticker)
    return ticker.history(period=period,interval=interval)

In [155]:
def engulfing(data, x_days = 1):
    results = []
    data['prev_candle_body_ratio'] = data['candle_body_ratio'].shift(1)
    data['prev_candle_color'] = data['candle_color'].shift(1)
    data['prev_candle_body_length'] = data['candle_body_length'].shift(1)
    data['prev_Close'] = data['Close'].shift(1)
    data['prev_Open'] = data['Open'].shift(1)
    data['prev_current_body_ratio'] = data['prev_candle_body_length'] / data['candle_body_length']
    return np.where(np.logical_and.reduce([data['prev_candle_body_ratio'] >= 15, data['prev_candle_color'] == "Red", data['candle_color'] == "Green", data['Close'] > data['prev_Open'], data['Open'] < data['prev_Close'], data['prev_current_body_ratio'] < .9]), "Bullish Engulfing", 
                   np.where(np.logical_and.reduce([data['prev_candle_body_ratio'] >= 15, data['prev_candle_color'] == "Green", data['candle_color'] == "Red", data['Open'] > data['prev_Close'], data['Close'] < data['prev_Open'], data['prev_current_body_ratio'] < .9]), "Bearish Engulfing", ""))

In [6]:
data = get_historical_stock_price(ticker, period = '1y', interval = '1d')

In [105]:
data['candle_body_length'] = (data['Close'] - data['Open']).abs()
data['candle_body_ratio'] = (data['candle_body_length'] * 100 / (data['High'] - data['Low']))
data['candle_length'] = data['High'] - data['Low']
data['candle_color'] = np.where(data['Open'] > data['Close'], "Red", "Green") 

In [156]:
data['engulf?'] = engulfing(data)

In [157]:
data[data['engulf?'] != ""]

,Open,High,Low,Close,Volume,Dividends,Stock Splits,candle_color,candle_body_length,candle_length,engulf?,candle_body_ratio,prev_candle_body_ratio,prev_candle_color,prev_candle_body_length,prev_Close,prev_Open,prev_current_body_ratio
Date,,,,,,,,,,,,,,,,,,
2021-09-27,1734.839613,1735.084820,1685.160648,1688.152222,5045337,0.0,0,Red,46.687391,49.924172,Bearish Engulfing,93.516607,16.455605,Green,5.737820,1730.033569,1724.295749,0.122899
2021-10-20,1794.326857,1812.570235,1759.654609,1767.795410,5876115,0.0,0,Red,26.531447,52.915625,Bearish Engulfing,50.139153,15.613088,Green,7.993773,1787.215942,1779.222169,0.301294
2022-01-14,1862.277695,1912.743244,1861.288175,1909.131470,7688506,0.0,0,Green,46.853774,51.455069,Bullish Engulfing,91.057645,17.768032,Red,8.114020,1876.922607,1885.036628,0.173178
2022-02-21,1683.174399,1719.094042,1673.773954,1712.563110,5120890,0.0,0,Green,29.388711,45.320088,Bullish Engulfing,64.846986,16.052889,Red,3.018086,1689.062012,1692.080098,0.102695
2022-02-28,1664.571449,1709.941063,1647.551743,1697.621460,15076542,0.0,0,Green,33.050011,62.389320,Bullish Engulfing,52.973828,39.882727,Red,20.186244,1676.841553,1697.027797,0.610779
2022-03-17,1903.144929,1903.144929,1813.840699,1835.164795,17731459,0.0,0,Red,67.980135,89.304230,Bearish Engulfing,76.121965,54.282649,Green,24.144325,1869.600098,1845.455773,0.355167
2022-03-22,1830.613001,1870.193823,1819.728275,1867.621094,5709382,0.0,0,Green,37.008092,50.465548,Bullish Engulfing,73.333381,17.357953,Red,7.866640,1833.631104,1841.497744,0.212565
2022-04-19,1619.498837,1619.498837,1533.756857,1545.631104,17044923,0.0,0,Red,73.867733,85.741980,Bearish Engulfing,86.151187,26.500041,Green,15.733401,1604.408691,1588.675290,0.212994
2022-04-29,1573.337736,1580.561284,1544.443783,1551.123047,9831395,0.0,0,Red,22.214689,36.117501,Bearish Engulfing,61.506716,28.410254,Green,12.467935,1566.015259,1553.547324,0.561247


In [143]:
# data['Close'].loc[:'2021-09-15']
(1668.045288 - 1656.716675) * .2 + 1668.045288

1670.3110106000001

In [ ]:
2/(span+1)
